In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py

import sys

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
####################################
#LOADING CLASSES

In [5]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [6]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="EntrainmentCalculation", dataName="Entrainment_Preprocessing",
                                dtype='int8')

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 inputParcelDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Proje

In [7]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [8]:
##############################
#SETUP

In [9]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [10]:
##############################################
#DATA LOADING FUNCTIONS

In [11]:
#INITIALIZE DATA FUNCTION
###############################################################
def InitiateArray(out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = ModelData.Ntime  # Number of timesteps
    if p_size is None:
        p_size = ModelData.Np  # Number of parcel indexes

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g','A_c','PROCESSED_A_g','PROCESSED_A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )
def InitiateArray_Job(Np,out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = ModelData.Ntime  # Number of timesteps
    if p_size is None:
        p_size = Np # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g','A_c','PROCESSED_A_g','PROCESSED_A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [12]:
#JOB ARRAY SETUP
def StartJobArray(num_jobs,job_id):
    total_elements=ModelData.Np #total num of variables
    
    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    # job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    # if job_id==0: job_id=1
    start_job, end_job = get_job_range(job_id)
    # print(f'\nstart_job = {start_job}, end_job = {end_job}')

    index_adjust=start_job
    return start_job,end_job,index_adjust
# num_jobs=1000; job_id=1
# [start_job,end_job,index_adjust] = StartJobArray(num_jobs,job_id)

In [13]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if ModelData.Np_str=='1e6': #1M parcels
    num_jobs=60  
    num_slurm_jobs=10 #this is the number that goes into sbatch script
elif ModelData.Np_str=='50e6': #50M parcels
    num_jobs=1200 
    num_slurm_jobs=150 #this is the number that goes into sbatch script
##############################

In [14]:
# MULTIPLE FILES

import os
import re
import pandas as pd
import xarray as xr
from glob import glob

def TestingTimes(files):
    print(files)
    for f in files:
        m = re.search(r'_(\d+-\d+-\d+)\.h5$', f)
        if m:
            print(m.group(1))
def OpenMultipleSingleTimes_LagrangianArray(directory, ModelData, pattern="Lagrangian_Binary_Array_*.h5"):
    """
    Load a sequence of Lagrangian .h5 files (each a single timestep)
    into one xarray.Dataset with dimensions (time, p),
    enforcing time order from ModelData.timeStrings.
    """
    # --- Find all available files
    files_all = glob(os.path.join(directory, pattern))
    if not files_all:
        raise FileNotFoundError(f"No files found in {directory} matching {pattern}")

    # --- Build the correctly ordered list according to ModelData.timeStrings
    files = []
    for t in ModelData.timeStrings:
        time_pattern = f"_{t}.h5"
        matched = [f for f in files_all if f.endswith(time_pattern)]
        if matched:
            files.append(matched[0])
        else:
            print(f"Missing file for time {t}")

    #####
    # TestingTimes(files) 
    #####
    
    # --- Convert ModelData.timeStrings (like ['0-00-00', '0-05-00', ...]) to pandas datetime
    #     using an arbitrary date
    times = pd.to_datetime([t.replace('-', ':') for t in ModelData.timeStrings], format="%H:%M:%S")

    # --- Open and concatenate along time
    ds = xr.open_mfdataset(
        files,
        engine="h5netcdf",
        phony_dims="sort",
        combine="nested",
        concat_dim="time",
    )

    # --- Rename the phony dimension to 'p'
    if "phony_dim_0" in ds.dims:
        ds = ds.rename({"phony_dim_0": "p"})

    # --- Assign your correct time coordinate
    ds = ds.assign_coords(time=times)

    return ds, files


directory = f"/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/LagrangianArrays/{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz/Lagrangian_Binary_Array/"

print("Loading Combined Lagrangian_Binary_Array")
Lagrangian_Binary_Array,files = OpenMultipleSingleTimes_LagrangianArray(directory, ModelData)
print("Done","\n")

Loading Combined Lagrangian_Binary_Array
Done 



In [15]:
# Reading Back Data Later
##############
def make_data_dict(var_names,start_job,end_job):

    data_dict = {var_name: Lagrangian_Binary_Array[var_name].isel(p=slice(start_job, end_job)).data.compute()
                 for var_name in var_names}
    return data_dict

In [16]:
def GetArrays(start_job,end_job):    
    var_names = ['A_g', 'A_c']
    data_dict = make_data_dict(var_names,start_job,end_job)
    A_g, A_c = (data_dict[k] for k in var_names)
    return A_g, A_c

In [17]:
################################
#FUNCTIONS

In [18]:
def extend_idxs(f,case):
    out=np.sort(np.add.outer(f, np.arange(case)).ravel())

    # #OLD METHOD (SLOW)
    # if np.any(f)==True:
    #     out=np.sort(np.concatenate([np.arange(idx, idx + case-1+1) for idx in f]))
    # else: 
    #     out=f
    return out

def find_sandwiched_patterns(changes, case):
    arr=changes
    
    window_size = case + 1  # e.g., for case=2, window_size = 3
    # The interior zeros count is (window_size - 2) which is case - 1
    pattern1 = np.array([-1] + [0]*(case - 1) + [1])
    pattern2 = np.array([1] + [0]*(case - 1) + [-1])
    # print(pattern1,pattern2)
    
    # Manually construct sliding windows
    windows = np.array([arr[i:i + window_size] for i in range(len(arr) - window_size + 1)])
    # print("Sliding windows:\n", windows) #TESTING
    
    #THE ALGORITHM
    turb_d=[]
    turb_e=[]
    count=0;max_iter=ModelData.Ntime;
    while np.any(((windows == pattern1) | (windows == pattern2)).all(axis=1)):
        count+=1; 
        if count>=max_iter: 
            print(count)
            break
        
        next_ind = np.where(((windows == pattern1) | (windows == pattern2)).all(axis=1))[0][0]
        
        if (windows[next_ind] == pattern1).all():
            turb_d.append(next_ind)
        elif (windows[next_ind] == pattern2).all(): 
            turb_e.append(next_ind) #append to list
    
        windows[0:next_ind+(case)+1,:] = 0 #removes from windows
    
    turb_d=np.array(turb_d,dtype=int); turb_e=np.array(turb_e,dtype=int)

    #EXTEND REST OF INDEXES TO PROCESS
    turb_d=extend_idxs(turb_d,case=case)
    turb_e=extend_idxs(turb_e,case=case)
    return turb_d,turb_e

In [20]:
# # TESTING
# changes = np.array([0,0,0,-1,1,0,0,-1,0,0,0,1,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=1) #<=1 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,1,0,0,-1,0,0,1,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=2) #<=2 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,0,1,0,0,0,0,1,0,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=3) #<=3 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,0,0,1,0,0,0,0,1,0,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=4) #<=4 in a row timesteps are removed
# print("Case matches at indices:", a,b)

# changes = np.array([0,0,0,-1,0,0,0,0,1,0,0,0,0,1,0,0,-1,0,0])
# [a,b] = find_sandwiched_patterns(changes, case=5) #<=5 in a row timesteps are removed
# print("Case matches at indices:", a,b)

In [22]:
###### (amount of time inside/outside of cloud to count as entrainment/detrainment)
mins_thresh=5 #5 mins
######

t_per_mins=1/((ModelData.time[1]-ModelData.time[0])/1e9/60).item() #timesteps per minute (<=1)
def get_changes(B):
    changes = np.diff(np.concatenate(([B[0]], B)))  # Add 0s to detect edges
    return changes
def PreProcessing(A,p):
    B = A[:,p] 

    # Find the changes in the array
    changes=get_changes(B)
    # print(f'B = {B}'); print(f'changes = {changes}') 

    #Determining the Case Number
    case=int(t_per_mins*mins_thresh) 
    
    if case>1:
        for case_ind in np.arange(case,0,-1): 
        # for case_ind in [case]:
            #Calling Algorithm and Correcting Parcel Data
            [turb_d,turb_e]=find_sandwiched_patterns(changes, case=case_ind)
            B[turb_d]=1
            B[turb_e]=0     
            changes=get_changes(B)
            # print(B)
    elif case==1:
        #Calling Algorithm and Correcting Parcel Data
        [turb_d,turb_e]=find_sandwiched_patterns(changes, case=case)
        B[turb_d]=1
        B[turb_e]=0
    return B

In [23]:
# #TESTING #CASE COUNTDOWN
# #shows the removal process (same for cases less than applicable case)
# B=np.array([1,1,1,1,0,0,0,1,1,1,1,0,0,0,1,0,0,1,1,1,0,0,0,1])
#                     #,#,#,        #,#,#       #,#,#
# print(B)

# #APPLYING
# changes=get_changes(B)
# [turb_d,turb_e]=find_sandwiched_patterns(changes,case=3)
# print(turb_d,turb_e)

# B[turb_d]=1
# B[turb_e]=0
# print(B)

# #APPLYING
# changes=get_changes(B)
# [turb_d,turb_e]=find_sandwiched_patterns(changes,case=2)
# print(turb_d,turb_e)

# B[turb_d]=1
# B[turb_e]=0
# print(B)

# #APPLYING
# changes=get_changes(B)
# [turb_d,turb_e]=find_sandwiched_patterns(changes,case=1)
# print(turb_d,turb_e)

# B[turb_d]=1
# B[turb_e]=0
# print(B)

In [24]:
#RUNNING
def Apply(A_g,A_c):
    PROCESSED_A_g = A_g.copy()
    PROCESSED_A_c = A_c.copy()
    for p in np.arange(Np):
        if np.mod(p,1e3)==0: print(f"{p}/{Np}")
        out1=PreProcessing(PROCESSED_A_g,p); PROCESSED_A_g[:,p]=out1
        out2=PreProcessing(PROCESSED_A_c,p); PROCESSED_A_c[:,p]=out2
    return PROCESSED_A_g,PROCESSED_A_c

#SAVING
def Save(Np,PROCESSED_A_g,PROCESSED_A_c,job_id):
    out_file=DataManager.outputDataDirectory+f'/processed_binary_arrays_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}_{job_id}.h5'
    print(f"Saving as {out_file}","\n")
    
    vars=['PROCESSED_A_g','PROCESSED_A_c']
    InitiateArray_Job(Np,out_file,vars,t_chunk_size=50,p_chunk_size=1000)
    
    with h5py.File(out_file, 'a') as f: 
        f['PROCESSED_A_g'][:]=PROCESSED_A_g
        f['PROCESSED_A_c'][:]=PROCESSED_A_c

In [25]:
####################################
#RUNNING

In [28]:
# %%time
########################################
#RUNNING
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=False) #if ISRUN is False, then will not run using slurm_job_array

print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 ==0: print(f"current job_id = {job_id}\n")
    [start_job,end_job,index_adjust]=StartJobArray(num_jobs,job_id)
    print(f'Running on Parcels {start_job}-{end_job}')
    # parcel=parcel1.isel(xh=slice(start_job,end_job)); 
    Np=end_job-start_job
    [A_g,A_c]=GetArrays(start_job,end_job)
    [PROCESSED_A_g,PROCESSED_A_c]=Apply(A_g,A_c)
    Save(Np,PROCESSED_A_g,PROCESSED_A_c,job_id)

    if not (np.allclose(A_g, PROCESSED_A_g) and np.allclose(A_c, PROCESSED_A_c)):
        print(f"found at job_id = {job_id} ==> start_job:end_job {start_job}:{end_job}")
        break

Running on Slurm_Jobs for Slurm_Job_Ids: (1, 60)
current job_id = 1

Running on Parcels 0-16667
0/16667
1000/16667
2000/16667
3000/16667
4000/16667
5000/16667
6000/16667
7000/16667
8000/16667
9000/16667
10000/16667
11000/16667
12000/16667
13000/16667
14000/16667
15000/16667
16000/16667
Saving as /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/1km_5min_34nz/Entrainment_Preprocessing/processed_binary_arrays_1km_5min_1e6_1.h5 

current job_id = 2

Running on Parcels 16667-33334
0/16667
1000/16667
2000/16667
3000/16667
4000/16667
5000/16667
6000/16667
7000/16667
8000/16667
9000/16667
10000/16667
11000/16667
12000/16667
13000/16667
14000/16667
15000/16667
16000/16667
Saving as /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/1km_5min_34nz/Entrainment_Preprocessing/processed_binary_arrays_1km_5min_1e6_2.h5 

current job_id = 3

Running

In [29]:
#COMBINING JOB_ARRAYS AFTER RUNNING
########################################################################
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
recombine=True 

In [30]:
# def Recombine(num_jobs):
#     dir2=dir+'Project_Algorithms/Entrainment/Processing_Out/'
#     dir3=dir+'Project_Algorithms/Entrainment/OUTPUT/'
#     out_file=dir3+f'processed_binary_arrays_{res}_{t_res}_{Np_str}.h5'
    
#     vars=['PROCESSED_A_g','PROCESSED_A_c']
#     InitiateArray(out_file,vars,t_chunk_size=50,p_chunk_size=100_000)
    
#     with h5py.File(out_file, 'r+') as f_out:
        
#         for job_id in np.arange(1,num_jobs+1):
#             if np.mod(job_id,5)==0: print(f"job_id = {job_id}")
#             [a,b,_] = StartJobArray(num_jobs,job_id)
        
#             in_file=DataManager.outputDataDirectory+f'processed_binary_arrays_{res}_{t_res}_{Np_str}_{job_id}.h5'
    
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][:,a:b]=f_in[var][:]

In [31]:
def Recombine_Dask(): #*#*
    #DASK-ENABLED
    import glob
    import re
    import dask
    from dask.diagnostics import ProgressBar
    
    def recombine_func(in_files,out_file):
        # matching_files = sorted(glob.glob(in_files))
        matching_files = sorted(
        glob.glob(in_files),
        key=lambda f: int(re.search(r'_(\d+)\.h5$', f).group(1))
    )
        
        print(f'recombining {len(matching_files)} files')
        print('Forming Dask Delayed Object')
        # out=xr.open_mfdataset(matching_files,engine='h5netcdf',concat_dim='phony_dim_1',combine='nested',phony_dims='sort')
        if ModelData.Np_str == '1e6':
            dim_1_length = 100_000
        elif ModelData.Np_str == '50e6':
            dim_1_length = 500_000
        chunks={'phony_dim_0': -1, 'phony_dim_1':dim_1_length}
        out = xr.open_mfdataset(
            matching_files,
            engine='h5netcdf',
            concat_dim='phony_dim_1',
            combine='nested',
            phony_dims='sort',
            chunks=chunks,
            parallel=True
        ) 
        print('RUNNING')
        with ProgressBar():
            delayed_write=out.to_netcdf(out_file, engine='h5netcdf', compute=False)
            delayed_write.compute()
        return out

    ####################################################################################
    in_files=DataManager.outputDataDirectory+f'/processed_binary_arrays_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}_*.h5'
    out_file=DataManager.outputDataDirectory+f'/processed_binary_arrays_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}.h5'
    out = recombine_func(in_files,out_file)
    print("\n",f"Saved to {out_file}")
    return out

In [34]:
if recombine==True:
    # Recombine(num_jobs=num_jobs)
    out=Recombine_Dask()

recombining 60 files
Forming Dask Delayed Object
RUNNING
[########################################] | 100% Completed | 2.94 sms

 Saved to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/1km_5min_34nz/Entrainment_Preprocessing/processed_binary_arrays_1km_5min_1e6.h5
